### Uncertainities associated with measured HI parameters

* Sytemic velocity: $V_{_{sys}}$
* Peak flux: $S_{_{peak}}$
* Integrated flux: $S_{_{int}}$
* Line widths: $W_{_{20}}$ and $W_{_{50}}$

#### 1.1 Jonga

\begin{eqnarray}
u \left( S_{_{peak}} \right) &=& \sqrt{\sigma^2_{_{rms}} + \left(\frac{S_{_{peak}}}{20} \right)^2}\hspace{4.7cm} \mathrm{(Barnes \, et \, al.\, 2001)}\\\\
u(S_{_{int}}) &=&\left(\frac{4}{SN}\right) \sqrt{\left(S_{_{peak}}S_{_{int}}\right) \delta v}\hspace{3.5cm} \mathrm{(Koribalski\, et \, al.\,  2004)}\\\\
u(V_{_{sys}})&=& \left(\frac{3}{SN}\right) \sqrt{\left(\frac{W_{_{20}}-W_{_{50}}}{2}\right) \delta v} \hspace{2.5cm} \mathrm{(Koribalski\, et \, al.\,  2004)}\\\\
u(W_{_{20}}) &=& 3 u(V_{_{sys}}) \,\,\,\,\,\& \,\,\,\,\, u(W_{_{50}})= 4 u(V_{_{sys}})  \hspace{2cm} \mathrm{(Schneider \,et\, al.\, 1986)}\\\\
\end{eqnarray}

In [1]:
from numpy import std,mean,pi,log,sqrt
from astropy.io import fits
hdr  = fits.getheader("/Volumes/THEMBA/Real/ppmos1/P_709:165:563.fits")


In [4]:
BMAJ,BMIN =  hdr["BMAJ"],hdr["BMIN"]
dv = hdr["cdelt3"]*1e-3      # channel width in km/s
cdelt = abs(hdr["cdelt1"])   # pixel size in rad
bmaj = BMAJ / cdelt          # beam major axis in pixels
bmin = BMIN / cdelt
H0   = 70.                   #km/s /Mpc
beam_area=4.0 * log(2.0) / (pi * bmaj * bmin)

In [5]:
def HI_MASS(v,f):
    flux= f *beam_area  #beam corrected flux [Jy km/s]
    d   = v/H0                                     #Distance in Mpc
    hi_mass = 2.36e+5*f* d**2
    HI_mass = log(hi_mass) / log(10.0)
    return (int(round(d)),round(flux,3),round(HI_mass,2)) #returns: distance,beam_correctd flux
                                                          #and log(HI mass)

In [162]:
def U_Speak(sigma,S_peak):
    x = sqrt(sigma**2 + (S_peak/20.)**2)
    return x

def U_Fint(SN,S_peak,S_int):
    x = (4./SN)*sqrt(S_peak*S_int*dv)
    return x

def U_Vsys(SN,w20,w50):
    x = (3./SN)*sqrt(dv*(w20-w50)/2.)
    return x     #units: km/s

def peak(name):
    demi = open(dir1+'linewidths/'+name+'.txt','r').readlines()
    f    = []
    for x in demi[6:]:
        d = x.split()
        e = float(d[-1])
        f.append(e)
    return (max(f),sum(f))  #units: Jy/beam


In [151]:
t=3
dir1  = "/Volumes/THEMBA/Real/ppmos"+str(t)+"/"
file0 = open(dir1+"pp"+str(t)+"_Raw","r").readlines()
#file1 = open(dir1+"pp"+str(t)+"_Table","w")
#file1.write("NO.    Id. name          Vel        W20       W50       Distance      F_peak      Flux_bc    HI Mass  \n")
cube_names = []

for line in file0[1:]:
    data   = line.split()
    flux   = float(data[8])
    w20    = float(data[7])
    w50    = float(data[6])
    #vel20  = float(data[5])
    vel    = float(data[4]) 
    SN     = float(data[2]) 
    d,f,mass= HI_MASS(vel,flux)
    cube_names.append(data[1])
#    pk     = peak(data[0])*1e3
#    file1.write(data[0]+"     "+data[1]+"     "+str(int(round(vel)))+"     "+str(int(round(w20)))+"     "
#                +str(int(round(w50)))+"      "+str(d)+"      "+str(round(pk,3))+"      "+str(round(f*1e3,3))+"      "+str(mass)+"\n")
    
#file1.close()
    

In [8]:
#vel,U_Vsys(SN,w20,w50)
#pk,U_Speak(0.4,pk)
#U_Fint(SN,pk*1e3,flux)
#flux

In [175]:
print name
name=cube_names[0]
pk,f_int=peak(name)
f_int*dv*beam_area

P_115175299


0.025723892167567597

In [56]:
dir3   = dir1+"linewidths/backup/"
hdu    =fits.open("/Volumes/THEMBA/Real/ppmos1/P_709:165:563.fits")
data   = hdu[0].data
header = hdu[0].header

In [101]:
import numpy as np
use = np.where(data>=2.87*0.4e-3)
data_inU = data[use]
(sum(data_inU)/11.6)*8.24

1.3588862826630217

In [103]:
data_inU

array([ 0.00126773,  0.00116618,  0.00131473, ...,  0.00115911,
        0.00127017,  0.00125239], dtype=float32)

In [126]:
ba=((pi/(4.*log(2)))*(23*16/6**2))

In [127]:
beam_area

0.081858066194635967

11.330900354567985

In [138]:
from astropy import units as u

In [140]:
a,b = 23*u.arcsec,16*u.arcsec
b.to(u.deg)

<Quantity 0.0044444444444444444 deg>